# Brain tumor 3D segmentation with MONAI

This tutorial shows how to construct a training workflow of multi-labels segmentation task.  
And it contains below features:
1. Transforms for TorchIO dictionary format data.
2. Define a new transform according to TorchIO, and MONAI transform API.
3. Load Nifti image with metadata, load a list of images and stack them.
5. 3D UNet model, Dice + cross entropy loss function, Mean Dice metric for 3D segmentation task.
6. Deterministic training for reproducibility.

The Brain tumor dataset can be downloaded from 
https://ipp.cbica.upenn.edu/ and  http://medicaldecathlon.com/.  

Target: Glioma segmentation tumour core, whole tumor, and enhancing tumor 
Modality: Multimodal multisite MRI data (FLAIR, T1w, T1gd,T2w)  
Size: 369 3D volumes   
Source: BRATS 2020 datasets.  
Challenge: Complex and heterogeneously-located targets

Below figure shows image patches with the tumor sub-regions that are annotated in the different modalities (top left) and the final labels for the whole dataset (right). (Figure taken from the [BraTS IEEE TMI paper](https://ieeexplore.ieee.org/document/6975210/))  
![image](https://ieeexplore.ieee.org/mediastore_new/IEEE/content/media/42/7283692/6975210/6975210-fig-3-source-large.gif)

The image patches show from left to right:
1. the whole tumor (yellow) visible in T2-FLAIR (Fig.A).
2. the tumor core (red) visible in T2 (Fig.B).
3. the enhancing tumor structures (light blue) visible in T1Gd, surrounding the cystic/necrotic components of the core (green) (Fig. C).
4. The segmentations are used to generate the final labels of the tumor sub-regions (Fig.D): edema (yellow), non-enhancing solid core (red), necrotic/cystic core (green), enhancing core (blue).

### TorchIO pipeline showing in the poster
The poster is taken from https://pytorch.org/ecosystem/pted/2021
![Poster](assets/torchIOPoster.png)
[To view](https://assets.pytorch.org/pted2021/posters/K4.png)

In [1]:
# Copyright 2020 MONAI Consortium
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#     http://www.apache.org/licenses/LICENSE-2.0
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import os
import shutil
import tempfile
import sys
import gc
import logging
import copy
import pdb

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
from sklearn.model_selection import KFold
from mpl_toolkits.axes_grid1 import ImageGrid
import scipy
from scipy import ndimage
import skimage
from skimage import morphology

import torch
import torch.nn as nn
import torch.nn.functional as F

import monai
from monai.transforms import (
    Activations,
    AsDiscrete,
    CastToTyped,
    Compose, 
    CropForegroundd,
    EnsureChannelFirstd, 
    LoadImaged,
    DivisiblePadd,
    Spacingd,
    NormalizeIntensity,
    NormalizeIntensityd,
    ScaleIntensity,
    RandCropByPosNegLabeld,
    ResizeWithPadOrCropd,
    RandFlipd, 
    RandGaussianNoised,
    RandGaussianSmoothd, 
    RandScaleIntensityd,
    RandZoomd,
    Resized,
    SpatialCrop, 
    SpatialPadd, 
    MapTransform,
    CastToType,
    ToTensord
)


from monai.config import KeysCollection
from monai.transforms.compose import MapTransform, Randomizable
from collections.abc import Iterable
from typing import Any, Dict, Hashable, Mapping, Optional, Sequence, Tuple, Union
from monai.utils import set_determinism
from monai.utils import (
    ensure_tuple,
    ensure_tuple_rep,
    ensure_tuple_size,
)

from monai.optimizers import LearningRateFinder

from monai.transforms.compose import MapTransform
from monai.transforms.utils import generate_spatial_bounding_box
from skimage.transform import resize
from monai.losses import DiceCELoss, DiceLoss
from monai.utils import set_determinism
from monai.inferers import sliding_window_inference

from nnunet.network_architecture.initialization import InitWeights_He
from monai.metrics import DiceMetric
import glob
import monai
from monai.metrics import compute_meandice
import random
import pickle
from collections import OrderedDict
from typing import Sequence, Optional
import SimpleITK as sitk

### monai and ignite based imports
import logging
import sys
from ignite.engine import Engine, Events
from ignite.contrib.handlers import FastaiLRFinder, ProgressBar
from ignite.engine import (
    Events,
    _prepare_batch,
    create_supervised_evaluator,
    create_supervised_trainer,
)
from ignite.handlers import EarlyStopping, ModelCheckpoint
import multiprocessing as mp
from tqdm import tqdm
import itkwidgets
from itkwidgets import view
import torchio as tio

from ranger21 import Ranger21
#from ranger.ranger2020 import Ranger as Ranger2020

monai.config.print_config()



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

MONAI version: 0.9.0
Numpy version: 1.22.3
Pytorch version: 1.10.1
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: af0e0e9f757558d144b655c63afcea3a4e0a06f5
MONAI __file__: /home/mmiv-ml/anaconda3/envs/sa_tumorseg22/lib/python3.9/site-packages/monai/__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.9
Nibabel version: 4.0.1
scikit-image version: 0.19.3
Pillow version: 9.0.1
Tensorboard version: NOT INSTALLED or UNKNOWN VERSION.
gdown version: NOT INSTALLED or UNKNOWN VERSION.
TorchVision version: 0.11.2
tqdm version: 4.64.0
lmdb version: NOT INSTALLED or UNKNOWN VERSION.
psutil version: 5.9.1
pandas ve

## Set deterministic training for reproducibility

In [4]:
seeds = 40961024
set_determinism(seed=seeds)
##np.random.seed(seeds) np random seed does not work here
!nvidia-smi

Thu Jun 30 22:11:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.172.01   Driver Version: 450.172.01   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-DGXS...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   48C    P0    59W / 300W |   9189MiB / 32505MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-DGXS...  On   | 00000000:08:00.0 Off |                    0 |
| N/A   

In [3]:
# MAX_THREADS =4
# sitk.ProcessObject.SetGlobalDefaultNumberOfThreads(MAX_THREADS)

### Here loading data of Brats2020 files from DataFrame

In [5]:
pd.set_option('display.max_colwidth', None)
data_rpath = '/home/mmiv-ml/data'
dataInferrpath = '/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData'
brainRMaskpath = '/raid/brats2021/T1wx4Brain_ROIs'
csvPath = '/home/mmiv-ml/saruarlive/tumorSeg21/assets/Brats21PlusMetaDF_14Jul21.csv'
dfBrats2021Training = pd.read_csv(csvPath)
dfBrats2021Training

,BraTS21ID,WTVol,TCVol,ETVol,EDVol,NECVol,WT_CoordX,WT_CoordY,WT_CoordZ,TC_CoordX,...,ED_CoordZ,NEC_CoordX,NEC_CoordY,NEC_CoordZ,t1wPath,t1cwPath,t2wPath,flairPath,segPath,is_train_test
0,BraTS2021_01483,107864,59424,0,48440,59424,85.491461,164.058147,77.561531,78.758768,...,72.053117,78.758768,165.909649,82.051764,BraTS2021_01483/BraTS2021_01483_t1.nii.gz,BraTS2021_01483/BraTS2021_01483_t1ce.nii.gz,BraTS2021_01483/BraTS2021_01483_t2.nii.gz,BraTS2021_01483/BraTS2021_01483_flair.nii.gz,BraTS2021_01483/BraTS2021_01483_seg.nii.gz,train
1,BraTS2021_01480,34700,10557,0,24143,10557,147.493343,126.876023,52.802853,145.425121,...,51.566624,145.425121,126.980676,55.630009,BraTS2021_01480/BraTS2021_01480_t1.nii.gz,BraTS2021_01480/BraTS2021_01480_t1ce.nii.gz,BraTS2021_01480/BraTS2021_01480_t2.nii.gz,BraTS2021_01480/BraTS2021_01480_flair.nii.gz,BraTS2021_01480/BraTS2021_01480_seg.nii.gz,train
2,BraTS2021_01350,25783,0,0,25783,0,165.052283,112.931854,56.507233,NaN,...,56.507233,NaN,NaN,NaN,BraTS2021_01350/BraTS2021_01350_t1.nii.gz,BraTS2021_01350/BraTS2021_01350_t1ce.nii.gz,BraTS2021_01350/BraTS2021_01350_t2.nii.gz,BraTS2021_01350/BraTS2021_01350_flair.nii.gz,BraTS2021_01350/BraTS2021_01350_seg.nii.gz,train
3,BraTS2021_01551,18958,0,0,18958,0,148.443190,185.222017,42.112934,NaN,...,42.112934,NaN,NaN,NaN,BraTS2021_01551/BraTS2021_01551_t1.nii.gz,BraTS2021_01551/BraTS2021_01551_t1ce.nii.gz,BraTS2021_01551/BraTS2021_01551_t2.nii.gz,BraTS2021_01551/BraTS2021_01551_flair.nii.gz,BraTS2021_01551/BraTS2021_01551_seg.nii.gz,train
4,BraTS2021_01439,18890,9329,0,9561,9329,78.750926,142.109952,55.188089,78.490728,...,53.995293,78.490728,140.208061,56.410548,BraTS2021_01439/BraTS2021_01439_t1.nii.gz,BraTS2021_01439/BraTS2021_01439_t1ce.nii.gz,BraTS2021_01439/BraTS2021_01439_t2.nii.gz,BraTS2021_01439/BraTS2021_01439_flair.nii.gz,BraTS2021_01439/BraTS2021_01439_seg.nii.gz,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246,BraTS2021_00806,136876,121765,81526,15111,40239,101.115309,103.301273,101.651656,101.769121,...,96.450930,101.767390,100.360844,106.028107,BraTS2021_00806/BraTS2021_00806_t1.nii.gz,BraTS2021_00806/BraTS2021_00806_t1ce.nii.gz,BraTS2021_00806/BraTS2021_00806_t2.nii.gz,BraTS2021_00806/BraTS2021_00806_flair.nii.gz,BraTS2021_00806/BraTS2021_00806_seg.nii.gz,valid
1247,BraTS2021_01430,256849,115924,67135,140925,48789,153.416097,132.299195,68.897909,152.365843,...,72.719908,152.548812,142.957654,65.067085,BraTS2021_01430/BraTS2021_01430_t1.nii.gz,BraTS2021_01430/BraTS2021_01430_t1ce.nii.gz,BraTS2021_01430/BraTS2021_01430_t2.nii.gz,BraTS2021_01430/BraTS2021_01430_flair.nii.gz,BraTS2021_01430/BraTS2021_01430_seg.nii.gz,valid
1248,BraTS2021_00060,240373,109066,61830,131307,47236,85.931353,130.302151,66.820737,87.874874,...,71.196943,86.609281,122.412440,61.499682,BraTS2021_00060/BraTS2021_00060_t1.nii.gz,BraTS2021_00060/BraTS2021_00060_t1ce.nii.gz,BraTS2021_00060/BraTS2021_00060_t2.nii.gz,BraTS2021_00060/BraTS2021_00060_flair.nii.gz,BraTS2021_00060/BraTS2021_00060_seg.nii.gz,valid
1249,BraTS2021_01385,146347,106252,70189,40095,36063,95.842142,86.124827,87.672135,97.039924,...,95.752513,99.370629,85.567340,84.666445,BraTS2021_01385/BraTS2021_01385_t1.nii.gz,BraTS2021_01385/BraTS2021_01385_t1ce.nii.gz,BraTS2021_01385/BraTS2021_01385_t2.nii.gz,BraTS2021_01385/BraTS2021_01385_flair.nii.gz,BraTS2021_01385/BraTS2021_01385_seg.nii.gz,valid


In [6]:
chnSeglst = ['t1wPath','t1cwPath', 't2wPath', 'flairPath', 'segPath']
for iname in chnSeglst:
    dfBrats2021Training[iname] = dfBrats2021Training[iname].apply(lambda x: glob.glob(os.path.join(dataInferrpath, x))[0])

dfBrats2021Training = dfBrats2021Training.sample(frac = 1.0, random_state = seeds)
display(dfBrats2021Training)

,BraTS21ID,WTVol,TCVol,ETVol,EDVol,NECVol,WT_CoordX,WT_CoordY,WT_CoordZ,TC_CoordX,...,ED_CoordZ,NEC_CoordX,NEC_CoordY,NEC_CoordZ,t1wPath,t1cwPath,t2wPath,flairPath,segPath,is_train_test
68,BraTS2021_01476,51173,23413,224,27760,23189,87.429015,119.159127,76.346413,86.156195,...,76.782601,86.219802,119.926431,75.855449,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01476/BraTS2021_01476_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01476/BraTS2021_01476_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01476/BraTS2021_01476_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01476/BraTS2021_01476_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01476/BraTS2021_01476_seg.nii.gz,train
843,BraTS2021_00818,146677,72231,46428,74446,25803,89.207326,159.983269,95.963007,86.028049,...,94.874701,85.567647,160.782700,98.102973,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00818/BraTS2021_00818_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00818/BraTS2021_00818_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00818/BraTS2021_00818_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00818/BraTS2021_00818_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00818/BraTS2021_00818_seg.nii.gz,train
868,BraTS2021_00809,182713,58722,51979,123991,6743,159.525168,132.562872,62.308030,162.423231,...,68.942254,162.284888,118.270058,43.627614,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00809/BraTS2021_00809_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00809/BraTS2021_00809_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00809/BraTS2021_00809_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00809/BraTS2021_00809_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00809/BraTS2021_00809_seg.nii.gz,train
443,BraTS2021_00730,78169,22314,17569,55855,4745,88.020980,137.273868,105.547020,88.801291,...,102.193179,88.137408,148.025079,114.029505,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00730/BraTS2021_00730_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00730/BraTS2021_00730_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00730/BraTS2021_00730_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00730/BraTS2021_00730_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00730/BraTS2021_00730_seg.nii.gz,train
102,BraTS2021_01556,41800,6656,3427,35144,3229,144.838493,127.864450,51.955933,144.545823,...,51.815246,144.028492,141.158253,52.869309,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01556/BraTS2021_01556_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01556/BraTS2021_01556_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01556/BraTS2021_01556_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01556/BraTS2021_01556_flair.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01556/BraTS2021_01556_seg.nii.gz,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
373,BraTS2021_00517,142604,13486,13165,129118,321,94.668431,152.921804,85.580068,101.276064,...,86.247951,100.074766,142.436137,77.383178,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00517/BraTS2021_00517_t1.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00517/BraTS2021_00517_t1ce.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00517/BraTS2021_00517_t2.nii.gz,/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00517/BraTS2021_00517_flair.nii.

In [7]:
t1wMask4save_folder = os.path.join('/raid/brats2021/T1wx2Brain_ROIs_BraTS21_Training')
brain_maskSaverT1wx2 = monai.data.NiftiSaver(output_dir=t1wMask4save_folder, output_postfix='BrainROIT1cwx2', output_ext='.nii.gz', resample=False, separate_folder=True)
brain_maskSaver = monai.data.NiftiSaver(output_dir=t1wMask4save_folder, output_postfix='BrainROI', output_ext='.nii.gz', resample=False, separate_folder=True)

In [8]:
brats21train_files = [{'image': (image_t1cw, image_t2)} \
                   for image_t1cw, image_t2 \
                   in zip(dfBrats2021Training['t1cwPath'], dfBrats2021Training['t2wPath'])]
print(len(brats21train_files))

1251


In [9]:
brats21train_files[20:25]

[{'image': ('/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01201/BraTS2021_01201_t1ce.nii.gz',
   '/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01201/BraTS2021_01201_t2.nii.gz')},
 {'image': ('/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01315/BraTS2021_01315_t1ce.nii.gz',
   '/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01315/BraTS2021_01315_t2.nii.gz')},
 {'image': ('/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01010/BraTS2021_01010_t1ce.nii.gz',
   '/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01010/BraTS2021_01010_t2.nii.gz')},
 {'image': ('/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01194/BraTS2021_01194_t1ce.nii.gz',
   '/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_01194/BraTS2021_01194_t2.nii.gz')},
 {'image': ('/raid/brats2021/RSNA_ASNR_MICCAI_BraTS2021_TrainingData/BraTS2021_00251/BraTS2021_00251_t1ce.nii.gz',
 

In [9]:
class CreateBrainMaskT1wx1d(MapTransform):
    
    
    def __init__(
        self,
        keys: KeysCollection,
        image_key = 'image',
    ) -> None:
        
        super().__init__(keys)
        self.image_key = image_key
        self.keys = ensure_tuple(keys)
    
    
    def __call__(self, data):
        d = dict(data)
        #ball_struct = morphology.ball(3)
        neighb_struct = np.ones((3,3,3))
        
        
        akey_meta_dict = f"{self.keys[0]}_meta_dict"
        d[self.keys[0]]=np.where(d[self.image_key] != 0, 1, 0).astype(np.uint8)
        
        #t1wBrainmask = ndimage.binary_closing(d[self.keys[0]][0], structure = neighb_struct).astype(np.uint8)
        t1wBrainmask = ndimage.binary_fill_holes(d[self.keys[0]][0], structure = neighb_struct).astype(np.uint8)
        #t1wBrainmask = t1wBrainmask[np.newaxis,:, :, :]
        t1wBrainmask = np.tile(t1wBrainmask, (2,1,1,1))
        
        
        d[self.keys[0]] = t1wBrainmask
        d[f"{self.keys[0]}_meta_dict"] = d[f"{self.image_key}_meta_dict"].copy()
             
        
        return d

In [10]:
class saveMask2file(MapTransform):
    
    
    def __call__(self, data):
        
        d = dict(data)
        
        for key in self.keys:

            
            input_file_path = d[f'{key}_meta_dict']['filename_or_obj']
            
            asub_id = os.path.basename(os.path.dirname(input_file_path))
            

        
            if not os.path.exists(t1wMask4save_folder):
                os.makedirs(t1wMask4save_folder)
                

            brain_mask_meta_dct = copy.deepcopy(d[f'{key}_meta_dict'])
            brain_mask_meta_dct['filename_or_obj'] = os.path.join(t1wMask4save_folder, f"{asub_id}.nii.gz")

            #pdb.set_trace()
            #breakpoint()
            brain_maskSaverT1wx2.save(data = d[key], meta_data = brain_mask_meta_dct)
            brain_maskSaver.save(data = d[key][0:1,:,:,:], meta_data = brain_mask_meta_dct)
            
            #pdb.set_trace()

            
            return d

In [11]:
keys = ["image"]
file_saveMask_transforms = Compose([
    LoadImaged(keys=keys, reader = "NibabelReader", allow_missing_keys=True),
    EnsureChannelFirstd(keys=keys),
    #CreateBrainMaskd(keys = "brain_mask", image_key = 'image'),
    CreateBrainMaskT1wx1d(keys = "brain_mask", image_key = 'image'),
    saveMask2file(keys = ["brain_mask"]),  
])
ds_brats21train_4SaveMask = monai.data.Dataset(data = brats21train_files, transform = file_saveMask_transforms) 


# for i in range(3):
#     print(val_ds_4SaveMask[i]['brain_mask'].shape)
    


def get_saveMasks(data_ds_ins):
    print(f"BraTS ID: {os.path.basename(os.path.dirname(data_ds_ins['brain_mask_meta_dict']['filename_or_obj']))}")
    return {'BraTSID': os.path.basename(os.path.dirname(data_ds_ins['brain_mask_meta_dict']['filename_or_obj']))}

In [12]:
ds_brats21train_4SaveMask[0]['brain_mask_meta_dict']

file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01476/BraTS2021_01476_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01476/BraTS2021_01476_BrainROI.nii.gz.


{'sizeof_hdr': array(348, dtype=int32),
 'extents': array(0, dtype=int32),
 'session_error': array(0, dtype=int16),
 'dim_info': array(0, dtype=uint8),
 'dim': array([  3, 240, 240, 155,   1,   1,   1,   1], dtype=int16),
 'intent_p1': array(0., dtype=float32),
 'intent_p2': array(0., dtype=float32),
 'intent_p3': array(0., dtype=float32),
 'intent_code': array(0, dtype=int16),
 'datatype': array(4, dtype=int16),
 'bitpix': array(16, dtype=int16),
 'slice_start': array(0, dtype=int16),
 'pixdim': array([1., 1., 1., 1., 0., 0., 0., 0.], dtype=float32),
 'vox_offset': array(0., dtype=float32),
 'scl_slope': array(nan, dtype=float32),
 'scl_inter': array(nan, dtype=float32),
 'slice_end': array(0, dtype=int16),
 'slice_code': array(0, dtype=uint8),
 'xyzt_units': array(2, dtype=uint8),
 'cal_max': array(0., dtype=float32),
 'cal_min': array(0., dtype=float32),
 'slice_duration': array(0., dtype=float32),
 'toffset': array(0., dtype=float32),
 'glmax': array(0, dtype=int32),
 'glmin': arra

### If not paralleled

In [13]:
for i in tqdm(range(len(ds_brats21train_4SaveMask))):
#for i in tqdm(range(50)):
    print(ds_brats21train_4SaveMask[i]['brain_mask'].shape)

  0%|                                                                                                                               | 1/1251 [00:01<37:30,  1.80s/it]

file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01476/BraTS2021_01476_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01476/BraTS2021_01476_BrainROI.nii.gz.
(2, 240, 240, 155)


  0%|▏                                                                                                                              | 2/1251 [00:03<38:40,  1.86s/it]

file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_00818/BraTS2021_00818_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_00818/BraTS2021_00818_BrainROI.nii.gz.
(2, 240, 240, 155)


  0%|▎                                                                                                                              | 3/1251 [00:05<39:33,  1.90s/it]

...

file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_00143/BraTS2021_00143_BrainROI.nii.gz.
(2, 240, 240, 155)
file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01241/BraTS2021_01241_BrainROIT1cwx2.nii.gz.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 1250/1251 [41:22<00:02,  2.24s/it]

file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01241/BraTS2021_01241_BrainROI.nii.gz.
(2, 240, 240, 155)
file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01396/BraTS2021_01396_BrainROIT1cwx2.nii.gz.


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1251/1251 [41:25<00:00,  1.99s/it]

file written: /raid/brats2021/T1wx2Brain_ROIs_BraTS21/BraTS2021_01396/BraTS2021_01396_BrainROI.nii.gz.
(2, 240, 240, 155)


## Creating masks from registered hd-bet masks

In [5]:
t1wMask4save_folder = os.path.join('/raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q')
brain_maskSaverT1wx2 = monai.data.NiftiSaver(output_dir=t1wMask4save_folder, output_postfix='BrainROIT1cwx2', output_ext='.nii.gz', resample=False, separate_folder=True)
brain_maskSaver = monai.data.NiftiSaver(output_dir=t1wMask4save_folder, output_postfix='BrainROI', output_ext='.nii.gz', resample=False, separate_folder=True)

In [6]:
t1cwHDMask4_folder = os.path.join('/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet')
t1cwHDMaskList  = sorted(glob.glob(os.path.join(t1cwHDMask4_folder, 'LGG-*/LGG-*_t1Gd_mask.nii.gz')))
t1cwHDMaskList.extend(glob.glob(os.path.join(t1cwHDMask4_folder, 'LGG-604/LGG-604_t1GdBrainROI.nii.gz*')))
t1cwHDMaskList

['/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-104/LGG-104_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-203/LGG-203_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-210/LGG-210_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-216/LGG-216_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-218/LGG-218_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-219/LGG-219_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-220/LGG-220_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-223/LGG-223_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q_BraTSLikeProcess_mnibet/LGG-225/LGG-225_t1Gd_mask.nii.gz',
 '/raid/brats2021/LGG_1p19q_rawNifti/LGG_1p19q

In [12]:
LGG_1p19qtrain_files = [{'image': image_hdt1cw} for image_hdt1cw in zip(t1cwHDMaskList)]
print(len(LGG_1p19qtrain_files))

159


In [13]:
class CreateBrainHDMaskT1wx1d(MapTransform):
    
    
    def __init__(
        self,
        keys: KeysCollection,
        image_key = 'image',
    ) -> None:
        
        super().__init__(keys)
        self.image_key = image_key
        self.keys = ensure_tuple(keys)
    
    
    def __call__(self, data):
        d = dict(data)
        #ball_struct = morphology.ball(3)
        neighb_struct = np.ones((3,3,3))
        
        
        akey_meta_dict = f"{self.keys[0]}_meta_dict"
        d[self.keys[0]]=np.where(d[self.image_key] == 1, 1, 0).astype(np.uint8)
        
        #t1wBrainmask = ndimage.binary_closing(d[self.keys[0]][0], structure = neighb_struct).astype(np.uint8)
        t1wBrainmask = ndimage.binary_fill_holes(d[self.keys[0]][0], structure = neighb_struct).astype(np.uint8)
        #t1wBrainmask = t1wBrainmask[np.newaxis,:, :, :]
        t1wBrainmask = np.tile(t1wBrainmask, (2,1,1,1))
       
        
        d[self.keys[0]] = t1wBrainmask
        d[f"{self.keys[0]}_meta_dict"] = d[f"{self.image_key}_meta_dict"].copy()
             
        
        return d

In [14]:
class saveHDMask2file(MapTransform):
    
    
    def __call__(self, data):
        
        d = dict(data)
        
        for key in self.keys:

            
            input_file_path = d[f'{key}_meta_dict']['filename_or_obj']
            
            asub_id = os.path.basename(os.path.dirname(input_file_path))
            
            if not os.path.exists(t1wMask4save_folder):
                os.makedirs(t1wMask4save_folder)
                

            brain_mask_meta_dct = copy.deepcopy(d[f'{key}_meta_dict'])
            brain_mask_meta_dct['filename_or_obj'] = os.path.join(t1wMask4save_folder, f"{asub_id}.nii.gz")

            
            #breakpoint()
            brain_maskSaverT1wx2.save(data = d[key], meta_data = brain_mask_meta_dct)
            brain_maskSaver.save(data = d[key][0:1,:,:,:], meta_data = brain_mask_meta_dct)
            
            #pdb.set_trace()

            
            return d

In [15]:
keys = ["image"]
file_saveMask_transforms = Compose([
    LoadImaged(keys=keys, reader = "NibabelReader", allow_missing_keys=True),
    EnsureChannelFirstd(keys=keys),
    #CreateBrainMaskd(keys = "brain_mask", image_key = 'image'),
    CreateBrainHDMaskT1wx1d(keys = "brain_mask", image_key = 'image'),
    saveHDMask2file(keys = ["brain_mask"]),  
])
ds_LGG_1p19train_4SaveMask = monai.data.Dataset(data = LGG_1p19qtrain_files, transform = file_saveMask_transforms) 


# for i in range(3):
#     print(val_ds_4SaveMask[i]['brain_mask'].shape)
    


def get_saveMasks(data_ds_ins):
    print(f"BraTS ID: {os.path.basename(os.path.dirname(data_ds_ins['brain_mask_meta_dict']['filename_or_obj']))}")
    return {'BraTSID': os.path.basename(os.path.dirname(data_ds_ins['brain_mask_meta_dict']['filename_or_obj']))}

In [16]:
ds_LGG_1p19train_4SaveMask[0]['brain_mask_meta_dict']

file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-104/LGG-104_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-104/LGG-104_BrainROI.nii.gz.


{'sizeof_hdr': array(348, dtype=int32),
 'extents': array(0, dtype=int32),
 'session_error': array(0, dtype=int16),
 'dim_info': array(0, dtype=uint8),
 'dim': array([  3, 240, 240, 155,   1,   1,   1,   1], dtype=int16),
 'intent_p1': array(0., dtype=float32),
 'intent_p2': array(0., dtype=float32),
 'intent_p3': array(0., dtype=float32),
 'intent_code': array(0, dtype=int16),
 'datatype': array(8, dtype=int16),
 'bitpix': array(32, dtype=int16),
 'slice_start': array(0, dtype=int16),
 'pixdim': array([1., 1., 1., 1., 0., 0., 0., 0.], dtype=float32),
 'vox_offset': array(0., dtype=float32),
 'scl_slope': array(nan, dtype=float32),
 'scl_inter': array(nan, dtype=float32),
 'slice_end': array(0, dtype=int16),
 'slice_code': array(0, dtype=uint8),
 'xyzt_units': array(2, dtype=uint8),
 'cal_max': array(0., dtype=float32),
 'cal_min': array(0., dtype=float32),
 'slice_duration': array(0., dtype=float32),
 'toffset': array(0., dtype=float32),
 'glmax': array(0, dtype=int32),
 'glmin': arra

In [17]:
for i in tqdm(range(len(ds_LGG_1p19train_4SaveMask))):
#for i in tqdm(range(50)):
    print(ds_LGG_1p19train_4SaveMask[i]['brain_mask'].shape)

  1%|█▍                                                                                                                                                                                                                               | 1/159 [00:01<04:31,  1.72s/it]

file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-104/LGG-104_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-104/LGG-104_BrainROI.nii.gz.
(2, 240, 240, 155)


  1%|██▊                                                                                                                                                                                                                              | 2/159 [00:03<04:30,  1.72s/it]

file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-203/LGG-203_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-203/LGG-203_BrainROI.nii.gz.
(2, 240, 240, 155)


  2%|████▏                                                                                                                                                                                                                            | 3/159 [00:05<04:26,  1.71s/it]

...

file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-660/LGG-660_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-660/LGG-660_BrainROI.nii.gz.
(2, 240, 240, 155)


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 158/159 [05:14<00:01,  1.82s/it]

file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-766/LGG-766_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-766/LGG-766_BrainROI.nii.gz.
(2, 240, 240, 155)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 159/159 [05:16<00:00,  1.99s/it]

file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-604/LGG-604_BrainROIT1cwx2.nii.gz.
file written: /raid/brats2021/LGG_1p19q_rawNifti/T1wx2Brain_ROIs_LGG_1p19q/LGG-604/LGG-604_BrainROI.nii.gz.
(2, 240, 240, 155)


## Checking shape

In [18]:
t1wMask4save_folder = os.path.join(data_rpath, 'T1wx4Brain_ROIs_TCGA_LGG_GBM')
TCGAProcSubjectsDF.insert(5, 'brain_maskPath', TCGAProcSubjectsDF['SubjectID'].apply(lambda x: os.path.join(t1wMask4save_folder, x, f"{x}_BrainROIT1wx4.nii.gz")))

segSave_folder = os.path.join(data_rpath, '3Ensemble_TCGA_LGG_GBM_Infer')
TCGAProcSubjectsDF.insert(6, 'segPath', TCGAProcSubjectsDF['SubjectID'].apply(lambda x: os.path.join(segSave_folder, x, f"{x}.nii.gz")))
#TCGAProcSubjectsDF.insert(6, 'segPath', TCGAProcSubjectsDF['t1cwPath'].apply(lambda x: glob.glob(f"{os.path.dirname(x)}/*_GlistrBoost_ManuallyCorrected.nii.gz")[0] if len(glob.glob(f"{os.path.dirname(x)}/*_GlistrBoost_ManuallyCorrected.nii.gz"))==1 else np.nan))


In [19]:
TCGAProcSubjectsDF = TCGAProcSubjectsDF.dropna(axis = 0)
TCGAProcSubjectsDF.reset_index(drop = True)

,SubjectID,t1wPath,t1cwPath,t2wPath,flairPath,brain_maskPath,segPath
0,TCGA-19-1388,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-19-1388/TCGA-19-1388_t1.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-19-1388/TCGA-19-1388_t1Gd.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-19-1388/TCGA-19-1388_t2.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-19-1388/TCGA-19-1388_flair.nii.gz,/raid/brats2021/TCGA_LGG_GBM/T1wx4Brain_ROIs_TCGA_LGG_GBM/TCGA-19-1388/TCGA-19-1388_BrainROIT1wx4.nii.gz,/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-19-1388/TCGA-19-1388.nii.gz
1,TCGA-06-0129,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-06-0129/TCGA-06-0129_t1.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-06-0129/TCGA-06-0129_t1Gd.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-06-0129/TCGA-06-0129_t2.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-06-0129/TCGA-06-0129_flair.nii.gz,/raid/brats2021/TCGA_LGG_GBM/T1wx4Brain_ROIs_TCGA_LGG_GBM/TCGA-06-0129/TCGA-06-0129_BrainROIT1wx4.nii.gz,/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-06-0129/TCGA-06-0129.nii.gz
2,TCGA-HT-7605,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-HT-7605/TCGA-HT-7605_t1.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-HT-7605/TCGA-HT-7605_t1Gd.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-HT-7605/TCGA-HT-7605_t2.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-HT-7605/TCGA-HT-7605_flair.nii.gz,/raid/brats2021/TCGA_LGG_GBM/T1wx4Brain_ROIs_TCGA_LGG_GBM/TCGA-HT-7605/TCGA-HT-7605_BrainROIT1wx4.nii.gz,/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-HT-7605/TCGA-HT-7605.nii.gz
3,TCGA-FG-8189,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-FG-8189/TCGA-FG-8189_t1.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-FG-8189/TCGA-FG-8189_t1Gd.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-FG-8189/TCGA-FG-8189_t2.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-FG-8189/TCGA-FG-8189_flair.nii.gz,/raid/brats2021/TCGA_LGG_GBM/T1wx4Brain_ROIs_TCGA_LGG_GBM/TCGA-FG-8189/TCGA-FG-8189_BrainROIT1wx4.nii.gz,/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-FG-8189/TCGA-FG-8189.nii.gz
4,TCGA-76-4926,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-76-4926/TCGA-76-4926_t1.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-76-4926/TCGA-76-4926_t1Gd.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-76-4926/TCGA-76-4926_t2.nii.gz,/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-76-4926/TCGA-76-4926_flair.nii.gz,/raid/brats2021/TCGA_LGG_GBM/T1wx4Brain_ROIs_TCGA_LGG_GBM/TCGA-76-4926/TCGA-76-4926_BrainROIT1wx4.nii.gz,/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-76-4926/TCGA-76-4926.nii.gz
...,...,...,...,...,...,...,...
258,TCGA-CS-6290,/raid/brats2021/TCGA_LGG_GBM/BraTS-Processed_TCGA_LGG_GBM/Pre-operative_TCGA_LGG_NIfTI_and_Segmentations_BraTS2017TestingData/TCGA-CS-6290/TCGA-CS-6290_2000.09.17_t1.nii.gz,/raid/brats2021/TCGA_LGG_GBM/BraTS-Processed_TCGA_LGG_GBM/Pre-operative_TCGA_LGG_NIfTI_and_Segmentations_BraTS2017TestingData/TCGA-CS-6290/TCGA-CS-6290_2000.09.17_t1Gd.nii.gz,/raid/brats2021/TCGA_LGG_GBM/BraTS-Processed_TCGA_LGG_GBM/Pre-operative_TCGA_LGG_NIfTI_and_Segmentations_BraTS2017TestingData/TCGA-CS-6290/TCGA-CS-6290_20

In [20]:
train_files = [{'image': image_nameT1ce, 'label': label_name, 'brain_mask':brain_mask} 
               for image_nameT1ce, label_name, brain_mask
               in zip(TCGAProcSubjectsDF['t1cwPath'],TCGAProcSubjectsDF['segPath'], TCGAProcSubjectsDF['brain_maskPath'])]  
train_files[0:10] 

[{'image': '/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-19-1388/TCGA-19-1388_t1Gd.nii.gz',
  'label': '/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-19-1388/TCGA-19-1388.nii.gz',
  'brain_mask': '/raid/brats2021/TCGA_LGG_GBM/T1wx4Brain_ROIs_TCGA_LGG_GBM/TCGA-19-1388/TCGA-19-1388_BrainROIT1wx4.nii.gz'},
 {'image': '/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-06-0129/TCGA-06-0129_t1Gd.nii.gz',
  'label': '/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-06-0129/TCGA-06-0129.nii.gz',
  'brain_mask': '/raid/brats2021/TCGA_LGG_GBM/T1wx4Brain_ROIs_TCGA_LGG_GBM/TCGA-06-0129/TCGA-06-0129_BrainROIT1wx4.nii.gz'},
 {'image': '/raid/brats2021/TCGA_LGG_GBM/TCGA_LGG_GBM_ManualSelect_BraTSLikeProcess_mnibet/TCGA-HT-7605/TCGA-HT-7605_t1Gd.nii.gz',
  'label': '/raid/brats2021/TCGA_LGG_GBM/3Ensemble_TCGA_LGG_GBM_Infer/TCGA-HT-7605/TCGA-HT-7605.nii.gz',
  'brain_mask': '/raid/brats2021/TCGA_LGG_

In [21]:
def threshold_at_one(x):
    # threshold at 1
    return x == 1

def threshold_ge_one(x):
    # threshold at 1
    return x >= 1

orig_patch_size = (240, 240, 155)
#SpatialPadd(keys=keys, spatial_size=patch_size),
keys = ["image", "label", "brain_mask"]

#pspac = (1.2, 1.2, 1.2)
# shape_check_transforms = Compose([
#     LoadImaged(keys=keys, reader = "NibabelReader"),
#     EnsureChannelFirstd(keys=keys),
#     CropForegroundd(keys=keys, source_key="brain_mask" ,select_fn=threshold_at_one),
#     #DivisiblePadd(keys = keys, k=2),
#     Spacingd(keys = keys, pixdim=pspac, mode = ('bilinear','nearest', 'nearest')),
#     #Resized(keys = keys, spatial_size = (128, 128, 128), mode = ('nearest','nearest', 'nearest')),
# ])

# shape_check_transforms2org = Compose([
#     LoadImaged(keys=keys, reader = "NibabelReader"),
#     EnsureChannelFirstd(keys=keys),
#     CropForegroundd(keys=keys, source_key="brain_mask" ,select_fn=threshold_at_one),
#     #DivisiblePadd(keys = keys, k=2),
#     Spacingd(keys = keys, pixdim=pspac, mode = ('bilinear','nearest', 'nearest')),
#     Spacingd(keys = keys, pixdim=(1, 1, 1), mode = ('bilinear','nearest', 'nearest')),
#     #Resized(keys = keys, spatial_size = (128, 128, 128), mode = ('nearest','nearest', 'nearest')),
# ])

iso_shape_check_transforms = Compose([
    LoadImaged(keys=keys, reader = "NibabelReader"),
    EnsureChannelFirstd(keys=keys),
    CropForegroundd(keys=keys, source_key="brain_mask", select_fn=threshold_at_one),
    CropForegroundd(keys=keys, source_key="label", select_fn=threshold_ge_one),
    #DivisiblePadd(keys = keys, k=2),
])

shape_check_ds = monai.data.Dataset(data = train_files, transform = iso_shape_check_transforms)

tumor_box_list = list()
for i in range(len(shape_check_ds)):

    aDSIns = shape_check_ds[i]
    print('Iso_CropFG:', aDSIns['image'].shape, 'label Iso_CropFG:', aDSIns['label'].shape)
    
    tumor_box_list.append(aDSIns['image'].shape)
    print(np.sum(aDSIns['label']==1), ' ### ', os.path.basename(aDSIns['label_meta_dict']['filename_or_obj']))


    print('#'*40)

Modifying image pixdim from [1. 1. 1. 1.] to [  1.           1.           1.         239.00209204]


Iso_CropFG: (1, 59, 96, 71) label Iso_CropFG: (1, 59, 96, 71)
1778  ###  TCGA-19-1388.nii.gz
########################################
Iso_CropFG: (1, 55, 82, 62) label Iso_CropFG: (1, 55, 82, 62)
61  ###  TCGA-06-0129.nii.gz
########################################
Iso_CropFG: (1, 37, 65, 44) label Iso_CropFG: (1, 37, 65, 44)
33  ###  TCGA-HT-7605.nii.gz
########################################
Iso_CropFG: (1, 62, 73, 71) label Iso_CropFG: (1, 62, 73, 71)
23085  ###  TCGA-FG-8189.nii.gz
########################################
Iso_CropFG: (1, 64, 77, 71) label Iso_CropFG: (1, 64, 77, 71)
38397  ###  TCGA-76-4926.nii.gz
########################################
Iso_CropFG: (1, 58, 73, 55) label Iso_CropFG: (1, 58, 73, 55)
2983  ###  TCGA-06-0147.nii.gz
########################################
Iso_CropFG: (1, 68, 92, 78) label Iso_CropFG: (1, 68, 92, 78)
7534  ###  TCGA-06-0645.nii.gz
########################################
Iso_CropFG: (1, 42, 80, 100) label Iso_CropFG: (1, 42, 80, 100)


In [22]:
stkArray = np.stack(tumor_box_list, axis = 0)


In [23]:
print(np.median(stkArray, axis = 0))
print(np.max(stkArray, axis = 0))
print(np.min(stkArray, axis = 0))
print(np.percentile(stkArray, 99, axis = 0))
stkArray[0]

[ 1. 58. 80. 66.]
[  1 124 168 109]
[ 1 18 20 24]
[  1.   110.42 132.76  96.9 ]


array([ 1, 59, 96, 71])

## Ground truth report

[ 1. 61. 84. 69.]

[  1 115 143 104]

[ 1 17 20 20]

[  1.  109.6 132.5 100. ]

array([ 1, 64, 82, 68])